In [3]:
!pip install langchain_openai langchain_google_genai langchain -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.1 which is incompatible.


In [19]:
import os
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
'''
or use these instead of ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_anthropic import ChatAnthropic
'''
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from keys import gemini_api_key

os.environ["GOOGLE_API_KEY"]= gemini_api_key

#llm= ChatOpenAI(temperature= 0, model= "gpt-3.5-turbo")
llm= ChatGoogleGenerativeAI(model= "gemini-1.5-pro-latest")

#template1
prompt= PromptTemplate(
    input_variables= ['x', 'y'],
    template= "what is {x} + {y}?"
)

chain= prompt | llm
result= chain.invoke(input= {"x": 55, "y": 42})
print(result.content)

55 + 42 = **97** 



## Prompt Templates

In [9]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

#template2
template= "Answer in one word. What {attribute} is the {input}?"
prompt_template= ChatPromptTemplate.from_template(template)
prompt= prompt_template.invoke({"attribute": "shape", "input": "moon"})
print(prompt)
print()
#template3
messages= [
    ("system", "You are an expert in science. Help the audience solve their issues. Give straight on to the point answers. Don't justify them."),
    ("human", "Which floats on top of the other? {input1} or {input2}?")
]
prompt_template2= ChatPromptTemplate.from_messages(messages)
prompt2= prompt_template2.invoke({"input1": "water", "input2": "oil"})
print(prompt2)

messages=[HumanMessage(content='Answer in one word. What shape is the moon?')]

messages=[SystemMessage(content="You are an expert in science. Help the audience solve their issues. Give straight on to the point answers. Don't justify them."), HumanMessage(content='Which floats on top of the other? water or oil?')]


In [21]:
from IPython.display import Markdown
messages= [
    SystemMessage(content= "Solve the following math problems"),
    HumanMessage(content= "What is 9 + 10?"),
    AIMessage(content= "19"),
    HumanMessage(content= "What is 10 + 3?")
]
result= llm.invoke(messages)
Markdown(result.content)

13 


In [23]:
#Real time chat application

chat_history= []

system_message= SystemMessage(content= "You are a helpful AI assistant")
chat_history.append(system_message)

while True:
    query= input("You: ")
    if query.lower()== "exit":
        break
    chat_history.append(HumanMessage(content= query))
    result= llm.invoke(chat_history)
    response= result.content
    chat_history.append(AIMessage(content= response))
    print("AI: {}".format(response))

You:  What is the largest desert?


AI: That depends on what kind of desert you're talking about! 

* **Largest hot desert:** The **Sahara Desert** in North Africa is the largest hot desert in the world, covering a massive 9.2 million square kilometers (3.6 million square miles). 

* **Largest desert overall:**  The **Antarctic Polar Desert** is actually the largest desert on Earth, encompassing 14 million square kilometers (5.4 million square miles) of ice and snow. Polar deserts are defined by their low precipitation, similar to hot deserts, but are much colder.

So, the Sahara wins for largest *hot* desert, but Antarctica takes the crown for the largest desert *overall*. 



You:  why did you assign a crown?


AI: That was just a figure of speech! I was trying to make the explanation a little more engaging. 😊 

I sometimes use metaphors and other literary devices to make my responses more interesting and easier to understand. 

Would you prefer I stick to a more straightforward, factual tone? I'm happy to adjust my communication style to your preference. 



You:  exit


In [35]:
print("Chat Message History")
chat_history

Chat Message History


[SystemMessage(content='You are a helpful AI assistant'),
 HumanMessage(content='What is the largest desert?'),
 AIMessage(content="That depends on what kind of desert you're talking about! \n\n* **Largest hot desert:** The **Sahara Desert** in North Africa is the largest hot desert in the world, covering a massive 9.2 million square kilometers (3.6 million square miles). \n\n* **Largest desert overall:**  The **Antarctic Polar Desert** is actually the largest desert on Earth, encompassing 14 million square kilometers (5.4 million square miles) of ice and snow. Polar deserts are defined by their low precipitation, similar to hot deserts, but are much colder.\n\nSo, the Sahara wins for largest *hot* desert, but Antarctica takes the crown for the largest desert *overall*. \n"),
 HumanMessage(content='why did you assign a crown?'),
 AIMessage(content="That was just a figure of speech! I was trying to make the explanation a little more engaging. 😊 \n\nI sometimes use metaphors and other li

## Chaining

### StrOutputParser: removes all the meta data from llm's output and returns only the content

In [21]:
from langchain.schema.output_parser import StrOutputParser

chain= prompt_template2 | llm | StrOutputParser()

result= chain.invoke({"input1": "water", "input2": "oil"})

print(result)

Oil floats on top of water. 



In [35]:
from langchain.schema.runnable import RunnableLambda
uppercase_op= RunnableLambda(lambda x: x.upper())
replace_space= RunnableLambda(lambda x: x.replace(" ", "*"))

chain= prompt_template2 | llm | StrOutputParser() | uppercase_op | replace_space
result= chain.invoke({"input1": "water", "input2": "oil"})
print(result)

OIL*FLOATS*ON*TOP*OF*WATER.*



#### Behind the scenes of the langchain chaining:<br>
RunnableSequence: sequence of runnables where the output of each is the input of the next.<br>
Implemented using | pipe operator.. both the sides of the pipe should be a Runnable

## Parallel Chaining

In [54]:
from langchain.schema.runnable import RunnableParallel

llm= ChatGoogleGenerativeAI(model= "gemini-1.5-pro-latest")

prompt= ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert product reviewer."),
        ("human", "List the main features of this product: {product}.")
    ]
)

def list_advs(features):
    advs_template= ChatPromptTemplate.from_messages(
        [
            ("system", "You are and expert product reviewer."),
            ("human", "Given these features: {features}, list their advantages")
        ]
    )
    return advs_template.format_prompt(features= features)

def list_dis_advs(features):
    dis_advs_template= ChatPromptTemplate.from_messages(
        [
            ("system", "You are and expert product reviewer."),
            ("human", "Given these features: {features}, list their disadvantages")
        ]
    )
    return dis_advs_template.format_prompt(features= features)

def combine(advs, dis_advs):
    return f"Advantages:\n{advs}\n\nDisadvantages:\n{dis_advs}"

branch1= RunnableLambda(lambda x: list_advs(x)) | llm | StrOutputParser()
branch2= RunnableLambda(lambda x: list_dis_advs(x)) | llm | StrOutputParser()

branch= RunnableParallel(branches= {"advs": branch1, "dis_advs": branch2})

concat= RunnableLambda(lambda x: combine(x["branches"]["advs"], x["branches"]["dis_advs"]))

chain= prompt | llm | StrOutputParser() | branch | concat

result= chain.invoke({"product": "HP Z-book firefly 14 g10; Processor: Intel Core i7-1280P, Graphics: Intel Iris Xe Graphics, RAM: 32GB, Storage: 256GB PCIe NVMe SSD, Display: 1920x1080 FHD"})

In [57]:
from IPython.display import Markdown
Markdown(result)

Advantages:
## Advantages of the HP ZBook Firefly 14 G10 based on your provided features:

Here are the advantages broken down by category:

**Performance:**

* **Strong Processor for Productivity:** The Intel Core i7-1280P offers a great balance of power and efficiency. You'll experience smooth performance in demanding applications and multitasking.
* **Generous RAM for Smooth Multitasking:** 32GB of RAM is more than enough for most professional workloads, allowing you to run multiple applications, browser tabs, and virtual machines simultaneously without slowdowns.
* **Fast Storage for Enhanced Responsiveness:** The 256GB PCIe NVMe SSD ensures rapid boot times, quick application loading, and snappy file transfers, significantly improving your overall productivity.

**Display & Design:**

* **Sharp and Vibrant Display:** The 1920x1080 FHD display provides good clarity and visual quality for productivity tasks, presentations, and media consumption.

**Other:**

* **ZBook Reliability and Durability:**  The ZBook series is known for its robust build quality, reliability, and ISV certifications, making it a dependable choice for professionals.
* **Portability for Professionals on the Go:**  The "Firefly" designation implies a lightweight and compact design, making it easy to carry and use while traveling or working remotely.

**Overall:**

This configuration of the HP ZBook Firefly 14 G10 offers a compelling package for professionals who prioritize a balance of performance and portability. It excels in everyday productivity tasks, multitasking, and handling demanding applications on the go. 


Disadvantages:
Here are the disadvantages of the HP ZBook Firefly 14 G10 configuration, based on your provided details:

* **Limited Storage:** The 256GB SSD, while fast, might be insufficient for professionals dealing with large files (e.g., high-resolution images, videos, complex datasets) or those who need ample local storage space. 
* **Insufficient Graphics for Demanding Tasks:** The integrated Intel Iris Xe Graphics, while suitable for everyday tasks and light creative work, will likely fall short for professionals requiring heavy graphical horsepower (e.g., CAD designers, video editors, 3D modelers). 
* **Display Resolution (Potentially):** While the 1920x1080 FHD resolution is decent for a 14-inch screen, some professionals might desire a higher resolution display (e.g., 1440p or 4K) for sharper visuals and more screen real estate, especially for tasks involving detailed work.

**In addition to the points you listed, here are some potential drawbacks to consider, depending on the specific use case and user preferences:**

* **Price:** ZBook models are often priced at a premium compared to consumer-grade laptops with similar specs.  
* **Battery Life:** While the processor is designed for efficiency, real-world battery life can vary significantly based on usage patterns, screen brightness, and other factors. Professionals needing extended unplugged use might find the battery life limiting.
* **Port Selection:**  The specific ports offered on this configuration are not mentioned.  Limited port selection could be a drawback for some professionals. 

It's important to note that these disadvantages might not be deal-breakers for every user. The overall value of this HP ZBook Firefly 14 G10 configuration depends heavily on the specific needs and priorities of the professional using it. 


## Chaining Branching

#### Difference between RunnableParallel and RunnableBranch'''

>parallel: all branches will be executed simultaneously<br>
>branchg: only one branch (or more, depending on a condition) is/are executed


In [64]:
from langchain.schema.runnable import RunnableBranch

DEFINE ALL THE PROMPTS
'''
positive_feedback= ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "Generate a thank you note for this positive feedback: {feedback}.")
    ]
)

negative_feedback= ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "Generate a response which addresses this negative feedback: {feedback}.")
    ]
)

neutral_feedback= ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "Generate a request for more details for this neutral feedback: {feedback}.")
    ]
)

escalate_feedback= ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "Generate a message to escalate this feedback to a human agent: {feedback}.")
    ]
)

main_prompt= ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "Classify the sentiment of this feedback as positive, negative, neutral or escalate: {feedback}")
    ]
)

#similiar to switch case; the last statement gets executed when all conditions fail
# (
#      condition ------------> if true, <this chain will execute>  #..other chains wont be executed
# )
branches= RunnableBranch(
    (
        lambda x: "positive" in x, positive_feedback | llm | StrOutputParser()
    ),
    (
        lambda x: "negative" in x, negative_feedback | llm | StrOutputParser()
    ),
    (
        lambda x: "neutral" in x, neutral_feedback | llm | StrOutputParser()
    ),
    escalate_feedback | llm | StrOutputParser()
    
)

chain= main_prompt| llm | StrOutputParser() | branches

result= chain.invoke({"feedback": "This product is terrible, Not worth the money I spent"})

In [66]:
Markdown(result)

## Addressing the Feedback: "This product is terrible. Not worth the money I spent."

We are incredibly sorry to hear about your negative experience with our product. We understand your frustration and disappointment, especially with you feeling like the product wasn't worth the investment. 

Your feedback is valuable to us, and we appreciate you taking the time to share your thoughts. To better understand the situation and see how we can help, could you please tell us more about what specifically you found terrible about the product? 

Knowing more about your experience will allow us to:

1. **Address your concerns directly:** We want to see if there's anything we can do to improve your experience with the product, whether it's offering troubleshooting advice, suggesting workarounds, or considering a refund.
2. **Improve our product:** Your feedback helps us identify areas where the product falls short and allows us to make necessary improvements for future customers.

We want to regain your trust and show you that we are committed to providing a positive experience with our products. Please feel free to contact us directly at [Contact Information] so we can discuss this further. 


## Retrieval Augmented Generation (RAG)

In [5]:
!pip install langchain_community -q

In [ ]:
!pip install sentence-transformers

In [5]:
!pip install langchain_huggingface -q

In [3]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [4]:
cur_dir= os.path.dirname(os.path.abspath("langchain-tutorial.ipynb"))

In [7]:
file_path= os.path.join(cur_dir, "color-psychology.txt")

In [9]:
persistent_directory= os.path.join(cur_dir, "db", "chroma_db")

In [11]:
if not os.path.exists(persistent_directory):
    if not os.path.exists(file_path):
        raise FileNotFoundError()
    loader= TextLoader(file_path)
    documents= loader.load() 

    text_splitter= CharacterTextSplitter(chunk_size= 200, chunk_overlap= 0)
    docs= text_splitter.split_documents(documents)

    db= Chroma.from_documents(
        docs, embeddings, 
        persist_directory= persistent_directory,   
        collection_metadata={"hnsw:space": "cosine"}
    )
else:
    print("Vector Store already exists")

Created a chunk of size 262, which is longer than the specified 200
Created a chunk of size 288, which is longer than the specified 200
Created a chunk of size 261, which is longer than the specified 200
Created a chunk of size 311, which is longer than the specified 200
Created a chunk of size 259, which is longer than the specified 200
Created a chunk of size 277, which is longer than the specified 200
Created a chunk of size 266, which is longer than the specified 200
Created a chunk of size 228, which is longer than the specified 200
Created a chunk of size 306, which is longer than the specified 200
Created a chunk of size 267, which is longer than the specified 200
Created a chunk of size 201, which is longer than the specified 200
Created a chunk of size 267, which is longer than the specified 200
Created a chunk of size 355, which is longer than the specified 200


In [13]:
len(docs)

26

In [15]:
docs[0]

Document(page_content='Color Psychology â€” as previously discussed, the world of graphic design and color theory are things that cannot be separated. Color theory is certainly very useful for creating good designs by paying attention to visual concepts that are attractive to the eye.', metadata={'source': 'C:\\Users\\HP\\Desktop\\chatbot\\color-psychology.txt'})

#### Similiarity searching: whichever chunk resembles most similiear to the query shows up on top..<br>(and later we take top k and give it to llm to make inference)

In [17]:
query= "How a sad event is depicted?."
retriever= db.as_retriever(
    search_type= "similarity_score_threshold",
    search_kwargs= {"k": 3, "score_threshold": 0.1}
)
relevant_docs= retriever.invoke(query);

In [29]:
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n {doc.page_content}\n")
    if doc.metadata:
        print(f"Source: {doc.metadata.get('source', 'Unknown')}\n")

Document 1:
 Showing a picture with a sad impression, a painter will give a little boredom to the picture. It can be seen from the paintings depicting a sad event, most of them have a lot of color, but there is little saturation, it could be said that the saturation is low.

Source: C:\Users\HP\Desktop\chatbot\color-psychology.txt

Document 2:
 That way, the colors used must also be appropriate and appropriate for the design created. We cannot possibly give a dominant black color to an image that shows a cheerful impression, nor can we give a bright dominant color such as blue, red, pink to an image that shows a sad impression.

Source: C:\Users\HP\Desktop\chatbot\color-psychology.txt

Document 3:
 So, what effect does black have on a personâ€™s psychology? What does the color black mean? Black gives a formal and exclusive impression. But on the other hand, black can also be interpreted as the color of darkness or sadness.

Source: C:\Users\HP\Desktop\chatbot\color-psychology.txt



In [9]:
persistent_directory= os.path.join(cur_dir, "db", "chroma_db_with_metadata")

#### Add metadata to the file (just change the meta data from "filepath" to "filename"

In [13]:
if not os.path.exists(persistent_directory):
    files= [f for f in os.listdir(cur_dir) if f.endswith(".txt")]
    documents= []
    for file in files:
        file_path= os.path.join(cur_dir, file)
        loader= TextLoader(file_path)
        docs= loader.load()
        for doc in docs:
            #here is the important part
            doc.metadata= {"source": file}
            documents.append(doc)
    text_splitter= CharacterTextSplitter(chunk_size= 1000, chunk_overlap= 0)
    docs= text_splitter.split_documents(documents)

    db= Chroma.from_documents(
        docs, embeddings, persist_directory= persistent_directory,
        collection_metadata={"hnsw:space": "cosine"}
    )
else:
    print("Vector store already exists")
    
        

#### Text Splitting

In [80]:
for file in files:
    file_path= os.path.join(cur_dir, file)
    print(file)

color-psychology-kfp.txt
color-psychology.txt


CharacterTextSplitter<br>
NLTKTextSplitter<br>
SpaCyTextSplitter<br>
SentenceTransformersTokenTextSplitter<br>
TextSplitter<br>
TokenTextSplitter<br>
RecursiveCharacterTextSplitter and many more...<br>

**the size of the chunk determines the no of characters to be inside the chunk**

In [19]:
query= "what is the color of the protagonist?"

retriever= db.as_retriever(
    search_type= "similarity",
    search_kwargs= {"k": 1}
)
relevant_docs= retriever.invoke(query)

In [21]:
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")

Document 1:
Gold is a color thatâ€™s present since the very beginning of the film, and codes for both our heroic characters/scenes, and for positive emotions in general, particularly their use to resolve a characterâ€™s progression from a state of struggling with things like regret and self-worth. It frequently shows up with another positive-coded color - green - which I will get into later.

Tigress against a golden background in the scene preceding Poâ€™s getting his ass handed to him by the Five, establishing her as a heroic/good guy character despite her ground-state mood of â€˜pissed offâ€™. The latter scene then shifts towards red when Shifu finally steps up to fight Po, possibly indicating Poâ€™s own power in refusing to quit, but also emphasizing just how pissed off this makes Shifu

Poâ€™s dream sequence, which is coded almost entirely gold and red, indicating Poâ€™s desire to become a powerful and respected kung-fu hero.



In [44]:
content= "\n\n".join([doc.page_content for doc in relevant_docs])

i_p= (
    '''Please answer the question only using the given content. If you don't know, answer that you don't know. Don't make up content and DO NOT use any other content other than what is provided.
    Content: {content}\n\n
    Question: {query}
    '''
)

prompt_template= ChatPromptTemplate.from_template(i_p)

chain= prompt_template | llm | StrOutputParser()

In [46]:
chain.invoke({"content": content, "query": query})

'The text states that gold is a color that codes for heroic characters, and that Po desires to become a kung fu hero. Therefore, the protagonist (Po) is associated with the color **gold**. \n'

# Conversation with RAG

In [49]:
import os
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import Chroma
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
llm= ChatGoogleGenerativeAI(model= "gemini-1.5-pro-latest")

database= "C:/Users/HP/Desktop/chatbot/db/chroma_db_with_metadata"

db= Chroma(persist_directory= database, 
           embedding_function= embeddings)

retriever= db.as_retriever(
    search_type= "similarity",
    search_kwargs= {"k": 3}
)

In [57]:
contextualize_q_system_prompt= (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just summarize and rephrase it if you can, otherwise return it as is." 
)

contextualize_q_prompt= ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

qa_system_prompt= (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, just say that you don't know. "
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

qa_prompt= ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain= create_stuff_documents_chain(llm, qa_prompt)

rag_chain= create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [65]:
def continual_chat():
    print("Chat with AI. Type 'exit' if you are done")
    chat_history= []
    while True:
        query= input("You: ")
        if query.lower() == "exit":
            break
        result= rag_chain.invoke({"input": query, "chat_history": chat_history})
        print(f"AI: {result['answer']}")
        chat_history.append(HumanMessage(content= query))
        chat_history.append(SystemMessage(content= result["answer"]))

In [71]:
continual_chat()

Start chatting with the AI! Type 'exit' to end the conversation.


You:  what color is bugatti?


AI: This document does not contain the answer to what color a Bugatti is. This document discusses color theory in relation to the movie Kung Fu Panda. 


You:  what color symbolises the villan?


ValueError: Unexpected message with type <class 'langchain_core.messages.system.SystemMessage'> at the position 2.

## Agents and Tools

In [84]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI

def get_current_time(*args, **kwargs):
    import datetime
    now= datetime.datetime.now()
    return now.strftime("%I:%M %p")

tools= [
    Tool(
        name= "Time",
        func= get_current_time,
        description= "Useful when you need to know the current time"
    )
]

prompt= hub.pull("hwchase17/react")
'''
Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
'''

agent= create_react_agent(
    llm= llm,
    tools= tools,
    prompt= prompt,
    stop_sequence= True
)

agent_executor= AgentExecutor.from_agent_and_tools(
    agent= agent,
    tools= tools,
    verbose= True
)

response= agent_executor.invoke({"input": "What time it is?"})



> Entering new AgentExecutor chain...
Question: What time it is?
Thought: I need to use the Time tool to get the current time.
Action: Time
Action Input: 05:05 PMQuestion: What time it is?
Thought: I need to use the Time tool to get the current time.
Action: Time
Action Input: 05:05 PMThought: I need to use the Time tool to get the current time.
Action: Time
Action Input: 05:05 PM

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


Question: What time it is?
Thought: I need to use the Time tool to get the current time.
Action: Time
Action Input: 05:05 PM

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


KeyboardInterrupt: 